# **COLONIAS**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    if nombre.startswith("df_") and "alfa" in nombre and "pachuca" in nombre 
]
# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_pachuca',
 'df_alfa_agosto_2024_pachuca',
 'df_alfa_febrero_2024_pachuca',
 'df_alfa_julio_2024_pachuca',
 'df_alfa_junio_2024_pachuca',
 'df_alfa_marzo_2024_pachuca',
 'df_alfa_mayo_2024_pachuca',
 'df_alfa_q_feb_2023_pachuca',
 'df_alfa_q_jul_2023_pachuca',
 'df_alfa_q_jun_2023_pachuca',
 'df_alfa_q_mar_2023_pachuca',
 'df_alfa_q_nov_2022_pachuca',
 'df_alfa_q_oct_2022_pachuca',
 'df_alfa_q_sep_2023_pachuca']

In [18]:
df_alfa_q_feb_2023_pachuca.columns

Index(['id', 'categoria', 'ids', 'propiedad', 'precio', 'tipo',
       'personas_interesadas', 'm2_total', 'm2_contruido',
       'precio_m2_contruido', 'precio_m2_terreno', 'promedio', 'baño',
       'medio_baño', 'baño_total', 'estacionamiento', 'recamaras',
       'antiguedad', 'publicado_hace', 'status'],
      dtype='object')

In [2]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'Precio':'precio','Colonia':'colonia'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [3]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
df_sin_colonia = []  # Lista para almacenar nombres de DataFrames sin la columna 'colonia'

for nombre_df in nombres_df_filtrados:
    df = globals()[nombre_df]
    
    # Verificar si el DataFrame contiene la columna 'colonia'
    if 'colonia' in df.columns:
        # Seleccionar las columnas 'precio' y 'colonia'
        segment_df = df[['precio', 'colonia']]
        # Añadir el DataFrame a la lista
        dataframes_list.append(segment_df)
    else:
        # Registrar el nombre del DataFrame que no tiene la columna 'colonia'
        df_sin_colonia.append(nombre_df)

# Imprimir los nombres de los DataFrames que no contienen la columna 'colonia'
if df_sin_colonia:
    print("Los siguientes DataFrames no tienen la columna 'colonia':")
    for df_name in df_sin_colonia:
        print(df_name)
else:
    print("Todos los DataFrames contienen la columna 'colonia'.")


Los siguientes DataFrames no tienen la columna 'colonia':
df_alfa_q_feb_2023_pachuca


In [4]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []

for nombre_df in nombres_df_filtrados:
    # Omitir el DataFrame específico 'df_alfa_q_feb_2023_pachuca'
    if nombre_df == 'df_alfa_q_feb_2023_pachuca':
        print(f"Omitiendo {nombre_df}.")
        continue  # Saltar a la siguiente iteración del bucle

    # Seleccionar las columnas 'precio' y 'colonia'
    segment_df = globals()[nombre_df][['precio', 'colonia']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

# Confirmar que el proceso ha completado
print("Proceso completado, se omitió 'df_alfa_q_feb_2023_pachuca'.")


Omitiendo df_alfa_q_feb_2023_pachuca.
Proceso completado, se omitió 'df_alfa_q_feb_2023_pachuca'.


## Más baratas

In [5]:
df_concatenado = pd.concat(dataframes_list)
    # Normalizar el nombre de la colonia
df_concatenado['colonia'] = df_concatenado['colonia'].str.lower().str.strip()

### Estandarizando las colonias 

In [6]:
# Convierte todos los valores a strings, maneja NaN o None como cadenas vacías
df_concatenado['colonia'] = df_concatenado['colonia'].astype(str)
# Convertir a minúsculas
df_concatenado['colonia'] = df_concatenado['colonia'].str.lower()

import unicodedata# Función para eliminar acentos
def eliminar_acentos(texto):
    # Normaliza y elimina los acentos
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    return ''.join(char for char in texto_normalizado if unicodedata.category(char) != 'Mn')
df_concatenado['colonia'] = df_concatenado['colonia'].apply(eliminar_acentos)

# Reemplazar Ñ
#df['propiedad'] = df['propiedad'].str.replace('ñ', 'n')
df_concatenado['colonia'] = df_concatenado['colonia'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
# Convertir la primera letra de cada palabra a mayúscula
df['colonia'] = df['colonia'].str.title()

In [7]:
# Diccionario de reemplazos
reemplazos = {
    'uni?n popular': 'unión popular',
    'uni??n Popular': 'unión popular',
    'uni??n popular': 'unión popular',
    'Unin Popular': 'unión popular',
    'unin popular':'unión popular',
    'lomas de nueva airosa.': 'lomas de nueva airosa',
    'josǹ l??pez portillo': 'josé lópez portillo',
    'El Atora3N': 'El Atorón','el atora3n': 'El Atorón'
}

# Reemplazar utilizando el diccionario
df_concatenado['colonia'] = df_concatenado['colonia'].replace(reemplazos)


In [8]:
    # Agrupar por colonia y calcular el promedio de precios
df_promedio_colonia = df_concatenado.groupby('colonia')['precio'].mean().reset_index()
    # Convertir la primera letra de cada palabra en la columna "colonia" a mayúscula
df_promedio_colonia['colonia'] = df_promedio_colonia['colonia'].str.title()
    # Ordenar por precio promedio en orden ascendente
df_promedio_colonia = df_promedio_colonia.sort_values(by='precio')

# Seleccionar las colonias más baratas
top_colonias_baratas = df_promedio_colonia.head(5)
top_colonias_baratas

,colonia,precio
608,Unión Popular,157136.0
282,La Surtidora,380000.0
10,El Atorón,444000.0
297,Lomas De Nueva Airosa,494153.5
625,Villa Elena,499550.0


In [9]:
colores = ['#df00ff','#939fad','#6773df','#67c2df','#7cc668','#ffb038','#ff4853','#3d45c0']
col = top_colonias_baratas['colonia']
precio = top_colonias_baratas['precio']
fig = go.Figure()
fig.add_trace(go.Bar(
    y=col[::-1],  # Invertir el orden de los datos
    x=precio[::-1],  # Invertir el orden de los datos
    orientation='h', 
    marker_color='#34a853',  # Colores de las barras
    text=precio[::-1].apply(lambda x: '${:,.2f}'.format(x)),  # Invertir el orden de los datos
    textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
))
fig.update_layout(
    #title='Colonias más baratas (Tulancingo)',  # Título de la gráfica
    xaxis=dict(
        title='$', # Título del eje x
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje X
        range=[0, 600000],  # Establecer el rango del eje y
        tickvals=[0, 200000,400000,600000],  # Definir los valores de las marcas en el eje y
        #ticktext=['$0.00', '$200,000.00', '$4,000,000.00','$6,000,000.00','$8,000,000.00'],  # Definir el texto de las marcas en el eje y
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom) 
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_colonias_baratas', carpeta='assets/graficas')
fig.show()

## Más caras

In [10]:
# Reemplazar "felipe Ãngeles" por "felipe angeles" después de la normalización
df_concatenado['colonia'] = df_concatenado['colonia'].replace('felipe ãngeles', 'felipe ángeles')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('felipe angeles', 'felipe ángeles')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('felipe ãngeles', 'felipe ángeles')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('felipe angeles', 'felipe ángeles')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('rinconada la concepci??n', 'rinconada la concepción')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('riinc?n del valle', 'rincón del valle')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('ricardo flores mag?n', 'ricardo flores magón')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('ricardo flores mag??n', 'ricardo flores magón')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('rinc?n de la plata', 'rincón de la plata')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('rinc??n del valle', 'rincón del valle')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('revoluci?n', 'revolución')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('parque urbanosan antonio el desmonte', 'parque urbano san antonio el desmonte')


In [11]:
# Agrupar por colonia y calcular el promedio de precios
df_promedio_colonia = df_concatenado.groupby('colonia')['precio'].mean().reset_index()
# Convertir la primera letra de cada palabra en la columna "colonia" a mayúscula
df_promedio_colonia['colonia'] = df_promedio_colonia['colonia'].str.title()
# Ordenar por precio promedio en orden descendente
df_promedio_colonia = df_promedio_colonia.sort_values(by='precio', ascending=False)

# Seleccionar las colonias más caras
top_colonias_caras = df_promedio_colonia.head(5)
top_colonias_caras

,colonia,precio
251,Jose Maria Sanchez Ramirez,4.552852e+07
193,Fracc Arboledas Santa Elena,2.422500e+07
196,Fracc Exhacienda De Coscotitlan,1.150000e+07
85,Campo De Golf,1.103906e+07
141,Dolores,1.000533e+07


In [12]:
df_promedio_colonia = df_promedio_colonia[~(df_promedio_colonia['colonia']=='Jose Maria Sanchez Ramirez')]
top_colonias_caras = df_promedio_colonia.head(5)
top_colonias_caras


,colonia,precio
193,Fracc Arboledas Santa Elena,2.422500e+07
196,Fracc Exhacienda De Coscotitlan,1.150000e+07
85,Campo De Golf,1.103906e+07
141,Dolores,1.000533e+07
237,Industrial La Paz,8.934529e+06


In [13]:
colores = ['#df00ff','#939fad','#6773df','#67c2df','#7cc668','#ffb038','#ff4853','#3d45c0']
col = top_colonias_caras['colonia']
precio = top_colonias_caras['precio']
fig = go.Figure()
fig.add_trace(go.Bar(
    y=col[::-1],  # Invertir el orden de los datos
    x=precio[::-1],  # Invertir el orden de los datos
    orientation='h', 
    marker_color='#ea4335',  # Colores de las barras
    text=precio[::-1].apply(lambda x: '${:,.2f}'.format(x)),  # Invertir el orden de los datos
    textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
))
fig.update_layout(
    #title='Colonias más caras (Tulancingo)',  # Título de la gráfica
    xaxis=dict(
        title='$', # Título del eje x
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje X
        #range=[0, 8000000],  # Establecer el rango del eje y
        #tickvals=[0, 2000000,4000000,6000000, 8000000],  # Definir los valores de las marcas en el eje y
        #ticktext=['$0.00', '$2,000,000.00', '$4,000,000.00','$6,000,000.00','$8,000,000.00'],  # Definir el texto de las marcas en el eje y
    ),   margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom) 
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_colonias_caras', carpeta='assets/graficas')
fig.show()

## Más predominantes

In [14]:
# Reemplazar "felipe Ãngeles" por "felipe angeles" después de la normalización
df_concatenado['colonia'] = df_concatenado['colonia'].replace('felipe ãngeles', 'felipe ángeles')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('felipe angeles', 'felipe ángeles')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('rinconada la concepci??n', 'rinconada la concepción')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('riinc?n del valle', 'rincón del valle')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('ricardo flores mag?n', 'ricardo flores magón')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('ricardo flores mag??n', 'ricardo flores magón')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('rinc?n de la plata', 'rincón de la plata')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('rinc??n del valle', 'rincón del valle')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('revoluci?n', 'revolución')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('parque urbanosan antonio el desmonte', 'parque urbano san antonio el desmonte')


In [15]:
# Calcular la frecuencia de cada colonia
frecuencia_colonia = df_concatenado['colonia'].value_counts().reset_index()
frecuencia_colonia.columns = ['colonia', 'total']
# Eliminar la entrada 'No obtenida' si está presente
frecuencia_colonia = frecuencia_colonia[frecuencia_colonia['colonia'] != 'No obtenida']
frecuencia_colonia = frecuencia_colonia[frecuencia_colonia['colonia'] != 'no obtenida']

# Calcular el total de ocurrencias en el DataFrame original
total_ocurrencias = frecuencia_colonia['total'].sum()

# Calcular el porcentaje para cada colonia
frecuencia_colonia['porcentaje'] = (frecuencia_colonia['total'] / total_ocurrencias) * 100

# Seleccionar el top 5 de colonias que más se repiten
top_5_colonias = frecuencia_colonia.head(5)

# Calcular el precio promedio de cada colonia
precio_promedio_colonia = df_concatenado.groupby('colonia')['precio'].mean().reset_index()

# Fusionar los DataFrames de frecuencia y precio promedio por colonia
colonias_predominantes = pd.merge(top_5_colonias, precio_promedio_colonia, on='colonia', how='left')
# Convertir la primera letra de cada palabra en la columna "colonia" a mayúscula
colonias_predominantes['colonia'] = colonias_predominantes['colonia'].str.title()
colonias_predominantes

,colonia,total,porcentaje,precio
0,Centro,1019,4.939409,1.225763e+06
1,El Tezontle,972,4.711585,6.468982e+06
2,San Antonio El Desmonte,883,4.280175,1.356585e+06
3,Valle Del Sol,865,4.192923,2.369541e+06
4,Zona Plateada,762,3.693650,9.988622e+05


In [16]:
fig = go.Figure()
porcentajes = colonias_predominantes['porcentaje']
colonias = colonias_predominantes['colonia']
# Agregar los datos como barras
fig.add_trace(go.Bar(
    x=colonias,  # Eje x: Modalidades
    y=porcentajes,  # Eje y: Porcentajes
     marker_color='#9500ff',   # Especifica los colores de las barras
    text=porcentajes,  # Texto que se mostrará en las barras (porcentaje)
    textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
    texttemplate='%{text:.1f}%',  # Formato del texto (porcentaje con un decimal)
))

# Personalizar el diseño de la gráfica
fig.update_layout(
    #title='Colonias Predominantes en Tulancingo',  # Título de la gráfica
    #xaxis=dict(title='%'),  # Título del eje x
    yaxis=dict(
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        #gridwidth=1,
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom) 
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_colonias_predominantes', carpeta='assets/graficas')
fig.show()